In [1]:
!pip install -q transformers>=4.38.2 peft==0.10.0 trl==0.8.1 protobuf==3.20.3 bitsandbytes accelerate datasets rouge-score

import pandas as pd
import torch
from datasets import Dataset
from tqdm import tqdm
import re

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 3.20.3 which is incompatible.
onnx 1.18.0 requires protobuf>=4.25.1, but you have protobuf 3.20.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 3.20.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which 

In [2]:
bias_list = """
Dưới đây là 10 kiểu sai lệch nhận thức:
1. All-or-Nothing Thinking – Nhìn nhận sự việc theo hai cực đối lập, không có mức trung gian.
2. Overgeneralization – Khái quát hóa quá mức từ một trải nghiệm tiêu cực.
3. Mental Filter – Chỉ tập trung vào chi tiết tiêu cực và bỏ qua bức tranh chung.
4. Disqualifying the Positive – Bác bỏ hoặc giảm giá trị những điều tích cực.
5. Jumping to Conclusions – Kết luận vội vàng dù không có bằng chứng.
6. Personalization – Cho rằng mọi việc đều liên quan đến bản thân.
7. Emotional Reasoning – Tin rằng cảm xúc tiêu cực phản ánh sự thật.
8. Should Statements – Dùng từ “phải/nên” để tự trách hoặc trách người khác.
9. Labeling and Mislabeling – Dán nhãn tiêu cực cho bản thân/người khác.
10. Magnification and Minimization – Phóng đại hoặc xem nhẹ tầm quan trọng của sự việc.
"""


In [3]:
train_csv = "/kaggle/input/cogdis-data/train.csv"
df = pd.read_csv(train_csv)

def convert_row(row):
    instruction = (
        f"Hãy phân tích đoạn hội thoại sau.\n"
        f"{bias_list}\n\n"
        "Yêu cầu:\n"
        "1) Đoạn này có sai lệch nhận thức không? (0 hoặc 1)\n"
        "2) Nếu có thì đó là loại nào (ghi tên tiếng Anh)?\n"
        "3) Giải thích rõ ràng.\n\n"
        f"Đoạn hội thoại:\n{row['dialogue']}"
    )

    output = (
        f"label: {row['label']}\n"
        f"bias_type: {row['bias_type']}\n"
        f"explanation: {row['explanation']}"
    )

    return {"instruction": instruction, "output": output}

processed = df.apply(convert_row, axis=1).tolist()
dataset = Dataset.from_list(processed)
dataset = dataset.train_test_split(test_size=0.05)


In [4]:
def format_chat(example):
    return {
        "text": (
            "<|im_start|>system\n"
            "Bạn là mô hình phân tích sai lệch nhận thức tiếng Việt.\n"
            "<|im_end|>\n"
            "<|im_start|>user\n"
            f"{example['instruction']}\n"
            "<|im_end|>\n"
            "<|im_start|>assistant\n"
            f"{example['output']}\n"
            "<|im_end|>"
        )
    }

dataset = dataset.map(format_chat)


Map:   0%|          | 0/6118 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

model_name = "Qwen/Qwen2.5-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    load_in_4bit=True,
)


2025-12-06 11:24:07.385839: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765020247.795988      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765020247.969773      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [6]:
lora = LoraConfig(
    r=32,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=[
        "q_proj","k_proj","v_proj","o_proj",
        "gate_proj","up_proj","down_proj"
    ],
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora)
model.print_trainable_parameters()


trainable params: 59,867,136 || all params: 3,145,805,824 || trainable%: 1.9030779186452418


In [7]:
training_args = TrainingArguments(
    output_dir="./qwen-bias-lora",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=50,
    save_steps=500,
    report_to="none", 
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    dataset_text_field="text",
    max_seq_length=2048,
)


Map:   0%|          | 0/6118 [00:00<?, ? examples/s]

Map:   0%|          | 0/322 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:322: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
trainer.train()

Step,Training Loss
50,0.843600
100,0.555500
150,0.521700
200,0.503400
250,0.510900
300,0.510900
350,0.499100
400,0.491800
450,0.483100
500,0.481700


TrainOutput(global_step=2295, training_loss=0.4426659270309415, metrics={'train_runtime': 15223.6193, 'train_samples_per_second': 1.206, 'train_steps_per_second': 0.151, 'total_flos': 1.5537401489896243e+17, 'train_loss': 0.4426659270309415, 'epoch': 3.0})

In [9]:
test_csv = "/kaggle/input/cogdis-data/test.csv"
df_test = pd.read_csv(test_csv)

def build_instruction(row):
    return (
        f"Hãy phân tích đoạn hội thoại sau.\n"
        f"{bias_list}\n\n"
        "Yêu cầu:\n"
        "1) Đoạn này có sai lệch nhận thức không? (0 hoặc 1)\n"
        "2) Nếu có thì đó là loại nào (tên tiếng Anh)?\n"
        "3) Giải thích rõ ràng.\n\n"
        f"Đoạn hội thoại:\n{row['dialogue']}"
    )

df_test["instruction"] = df_test.apply(build_instruction, axis=1)


In [10]:
def make_prompt(inst):
    return (
        "<|im_start|>system\n"
        "Bạn là mô hình phân tích sai lệch nhận thức tiếng Việt.\n"
        "<|im_end|>\n"
        "<|im_start|>user\n"
        f"{inst}\n"
        "<|im_end|>\n"
        "<|im_start|>assistant\n"
    )


In [11]:
def generate_answer(prompt):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    out = model.generate(
        **inputs,
        max_new_tokens=256,
        temperature=0.2
    )
    return tokenizer.decode(out[0], skip_special_tokens=True)

def parse_output(text):
    label = None; bias = None; expl = ""

    m = re.search(r"label\s*:\s*(\d)", text)
    if m: label = int(m.group(1))

    m = re.search(r"bias_type\s*:\s*(.+)", text)
    if m: bias = m.group(1).strip()

    m = re.search(r"explanation\s*:\s*(.+)", text, re.S)
    if m: expl = m.group(1).strip()

    return label, bias, expl


In [12]:
pred_lbl = []
pred_bias = []
pred_exp = []

for inst in tqdm(df_test["instruction"]):
    prompt = make_prompt(inst)
    gen = generate_answer(prompt)
    pl, pb, pe = parse_output(gen)

    pred_lbl.append(pl)
    pred_bias.append(pb)
    pred_exp.append(pe)

df_test["pred_label"] = pred_lbl
df_test["pred_bias"] = pred_bias
df_test["pred_explanation"] = pred_exp


100%|██████████| 725/725 [2:15:13<00:00, 11.19s/it]


In [13]:
label_acc = (df_test["label"] == df_test["pred_label"]).mean()
print("Label Accuracy:", label_acc)


Label Accuracy: 0.9848275862068966


In [14]:
mask = df_test["label"] == 1
bias_acc = (df_test[mask]["bias_type"] == df_test[mask]["pred_bias"]).mean()
print("Bias Type Accuracy:", bias_acc)


Bias Type Accuracy: 0.8922155688622755


In [15]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)

scores = []
for ref, pred in zip(df_test["explanation"], df_test["pred_explanation"]):
    score = scorer.score(ref, pred)["rougeL"].fmeasure
    scores.append(score)

print("ROUGE-L:", sum(scores)/len(scores))


ROUGE-L: 0.5503542616611636


In [16]:
df_test.to_csv("test_predictions.csv", index=False)
print("Đã lưu test_predictions.csv")


Đã lưu test_predictions.csv


In [17]:
save_path = "./qwen-bias-lora-final"

trainer.model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Đã lưu mô hình LoRA tại: {save_path}")

Đã lưu mô hình LoRA tại: ./qwen-bias-lora-final


In [18]:
!zip -r /kaggle/working/qwen-bias-lora-final.zip /kaggle/working/qwen-bias-lora-final

  adding: kaggle/working/qwen-bias-lora-final/ (stored 0%)
  adding: kaggle/working/qwen-bias-lora-final/adapter_model.safetensors (deflated 7%)
  adding: kaggle/working/qwen-bias-lora-final/chat_template.jinja (deflated 71%)
  adding: kaggle/working/qwen-bias-lora-final/added_tokens.json (deflated 67%)
  adding: kaggle/working/qwen-bias-lora-final/README.md (deflated 66%)
  adding: kaggle/working/qwen-bias-lora-final/adapter_config.json (deflated 54%)
  adding: kaggle/working/qwen-bias-lora-final/tokenizer_config.json (deflated 89%)
  adding: kaggle/working/qwen-bias-lora-final/merges.txt (deflated 57%)
  adding: kaggle/working/qwen-bias-lora-final/vocab.json (deflated 69%)
  adding: kaggle/working/qwen-bias-lora-final/special_tokens_map.json (deflated 69%)


In [19]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('qwen-bias-lora-final.zip')

/kaggle/working


/kaggle/working/qwen-bias-lora-final.zip